In [2]:
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer

stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 
             'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 
             'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
             'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 
             'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 
             'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 
             'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 
             'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 
             'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 
             'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 
             'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
             ]

df = pd.read_pickle('jiujitsu videos links.pkl')
df

,title,view_num,url
1106,Two WORST Martial Arts Techniques Ever!,3.71e+06,https://www.youtube.com/watch?v=kvTxH11O2HU
933,The Most Dangerous Takedown in Judo & BJJ,2.7e+06,https://www.youtube.com/watch?v=XRklGgwpbj0
2344,Death of the Cauliflower Ear,2.3e+06,https://www.youtube.com/watch?v=AzZWp1oPKo0
1083,Five Triangle Chokes You Must Know,1.86e+06,https://www.youtube.com/watch?v=FbXunEbkf8E
916,2 Easy Takedowns for People Who Don't Do Taked...,1.72e+06,https://www.youtube.com/watch?v=8uaFGdlYwOw
...,...,...,...
4554,#GreyBelt #TechnicalStance,55,https://www.youtube.com/watch?v=U0qKDLzVjgo
4671,Gavin Quick NoGi 6/15/19,49,https://www.youtube.com/watch?v=kr107vrDgTs
5054,"Gent, Belgium 2015",46,https://www.youtube.com/watch?v=jrUCVry9tBU
5256,My boy Gavin,37,https://www.youtube.com/watch?v=LVsceYlhpc4


In [12]:
# tokenization
text = df[['title']].copy()
text['tokenized_title'] = text.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)
text['tokenized_title'] = text['tokenized_title'].apply(lambda x: [word.lower() for word in x if word.lower() not in (stopwords)])
text['tokenized_title'] = text['tokenized_title'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 단어 인덱싱을 위한 단어 딕셔너리
words_list = {}
i=0
for i, row in text.iterrows():
    for word in row['tokenized_title']:
        if word not in words_list:
            words_list[word] = i 
            i += 1
detokenized_doc = []
for i, row in text.iterrows():
    t = ' '.join(row['tokenized_title'])
    detokenized_doc.append(t)
text['detokenized_title'] = detokenized_doc
text = text.drop(['title'], axis=1)
new_df = pd.concat([df, text], axis=1)
new_df.to_csv('tokenized title.csv')

In [13]:
new_df

,title,view_num,url,tokenized_title,detokenized_title
1106,Two WORST Martial Arts Techniques Ever!,3.71e+06,https://www.youtube.com/watch?v=kvTxH11O2HU,"[two, worst, martial, arts, techniques, ever, !]",two worst martial arts techniques ever !
933,The Most Dangerous Takedown in Judo & BJJ,2.7e+06,https://www.youtube.com/watch?v=XRklGgwpbj0,"[dangerous, takedown, judo, &, bjj]",dangerous takedown judo & bjj
2344,Death of the Cauliflower Ear,2.3e+06,https://www.youtube.com/watch?v=AzZWp1oPKo0,"[death, cauliflower, ear]",death cauliflower ear
1083,Five Triangle Chokes You Must Know,1.86e+06,https://www.youtube.com/watch?v=FbXunEbkf8E,"[five, triangle, choke, must, know]",five triangle choke must know
916,2 Easy Takedowns for People Who Don't Do Taked...,1.72e+06,https://www.youtube.com/watch?v=8uaFGdlYwOw,"[2, easy, takedowns, people, n't, takedowns]",2 easy takedowns people n't takedowns
...,...,...,...,...,...
4554,#GreyBelt #TechnicalStance,55,https://www.youtube.com/watch?v=U0qKDLzVjgo,"[#, greybelt, #, technicalstance]",# greybelt # technicalstance
4671,Gavin Quick NoGi 6/15/19,49,https://www.youtube.com/watch?v=kr107vrDgTs,"[gavin, quick, nogi, 6/15/19]",gavin quick nogi 6/15/19
5054,"Gent, Belgium 2015",46,https://www.youtube.com/watch?v=jrUCVry9tBU,"[gent, ,, belgium, 2015]","gent , belgium 2015"
5256,My boy Gavin,37,https://www.youtube.com/watch?v=LVsceYlhpc4,"[boy, gavin]",boy gavin


In [3]:
title_list = text['detokenized_title'].to_list()
vect = TfidfVectorizer(ngram_range=(1,3))
vect.fit_transform(title_list)
vocab_dict = vect.vocabulary_
vocab_dict = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab_df = pd.DataFrame(vocab_dict)
vocab_df.to_csv('countvectorizer.csv')
vocab_df
# stop_list = []
# for voca in vocab_dict:
#     if voca[0] in stopwords:
#         stop_list.append(voca)

,0,1
0,zwevegem belgium,35185
1,zwevegem,35184
2,zorro pass,35183
3,zorro,35182
4,zombie apocalypse,35181
...,...,...
35181,000 miles arctic,4
35182,000 miles,3
35183,000 giveaway chance,2
35184,000 giveaway,1


In [22]:
# for i, row in new_df.iterrows():
#     tokens = row['tokenized_title']
#     if 'guard' in tokens:

#  Positions
guard_ = new_df[new_df['tokenized_title'].apply(lambda x: 'guard' in x and 'pass' not in x)].copy()
side_ = new_df[new_df['tokenized_title'].apply(lambda x: 'side' in x)].copy()
top_ = new_df[new_df['tokenized_title'].apply(lambda x: 'top' in x)].copy()
north_south_ = new_df[new_df['tokenized_title'].apply(lambda x: 'north' in x or 'south' in x)].copy()
back_ = new_df[new_df['tokenized_title'].apply(lambda x: 'back' in x)].copy()

# Transitions
pass_ = new_df[new_df['tokenized_title'].apply(lambda x: 'pass' in x)].copy()
sweep_ = new_df[new_df['tokenized_title'].apply(lambda x: 'sweep' in x)].copy()

# Escapes
escape_ = new_df[new_df['tokenized_title'].apply(lambda x: 'escape' in x)].copy()

# Submissions
submission_ = new_df[new_df['tokenized_title'].apply(lambda x: 'submission' in x)].copy()
# Arm-Locks
americana_ = new_df[new_df['tokenized_title'].apply(lambda x: 'americana' in x)].copy()
kimura_ = new_df[new_df['tokenized_title'].apply(lambda x: 'kimura' in x)].copy()
armbar_ = new_df[new_df['tokenized_title'].apply(lambda x: 'arm' in x and 'bar' in x)].copy()
omoplata_ = new_df[new_df['tokenized_title'].apply(lambda x: 'omoplata' in x)].copy()

# Chokes
choke_ = new_df[new_df['tokenized_title'].apply(lambda x: 'choke' in x)].copy()
collar_choke_ = new_df[new_df['tokenized_title'].apply(lambda x: 'collar' in x and 'choke' in x)].copy()
guillotine_choke_ = new_df[new_df['tokenized_title'].apply(lambda x: 'guill' in x)].copy()
triangle_choke_ = new_df[new_df['tokenized_title'].apply(lambda x: 'triangle' in x)].copy()

# Leg-Locks
ankle_lock_ = new_df[new_df['tokenized_title'].apply(lambda x: 'ankle' in x and 'lock' in x)].copy()
knee_bar_ = new_df[new_df['tokenized_title'].apply(lambda x: 'knee' in x and 'bar' in x)].copy()
heel_hook_ = new_df[new_df['tokenized_title'].apply(lambda x: 'heel' in x and 'hook' in x)].copy()

# Solo-Drills
drill_ = new_df[new_df['tokenized_title'].apply(lambda x: 'drill' in x)].copy()



In [28]:
escape_

,title,view_num,url,tokenized_title,detokenized_title
994,How to Escape Kesa Gatame and the Headlock,1.13e+06,https://www.youtube.com/watch?v=9Im8IKc8NeY,"[escape, kesa, gatame, headlock]",escape kesa gatame headlock
1079,BJJ Self Defense Lesson 2: Escaping the Mount,1.03e+06,https://www.youtube.com/watch?v=PJz8YQ5rXZw,"[bjj, self, defense, lesson, 2, :, escape, mount]",bjj self defense lesson 2 : escape mount
1002,How to Escape the Triangle Choke from Guard,950000,https://www.youtube.com/watch?v=szAvJFvIzx4,"[escape, triangle, choke, guard]",escape triangle choke guard
1060,A Mount Escape that Works Against Larger Oppon...,820000,https://www.youtube.com/watch?v=q2OAJF__P2w,"[mount, escape, work, larger, opponents]",mount escape work larger opponents
3867,Side control escape against tight top pressure,780000,https://www.youtube.com/watch?v=VBT67Grv28k,"[side, control, escape, tight, top, pressure]",side control escape tight top pressure
...,...,...,...,...,...
4542,#GreyBelt #Kimura escape,177,https://www.youtube.com/watch?v=_09RYkUlsA0,"[#, greybelt, #, kimura, escape]",# greybelt # kimura escape
4541,#GreyBelt #Guillotine escapes,147,https://www.youtube.com/watch?v=AmXhLj0s0-Y,"[#, greybelt, #, guillotine, escape]",# greybelt # guillotine escape
4546,#GreyBelt Bow and Arrow & Escape,122,https://www.youtube.com/watch?v=Iwomu9d-oNE,"[#, greybelt, bow, arrow, &, escape]",# greybelt bow arrow & escape
4551,#GreyBelt #EzekielChoke escape,91,https://www.youtube.com/watch?v=s62qAZxMHAA,"[#, greybelt, #, ezekielchoke, escape]",# greybelt # ezekielchoke escape
